In [1]:
import pandas as pd
import numpy as np
from SPARQLWrapper import SPARQLWrapper, JSON

In [16]:
EC2_URI = 'ec2-18-217-207-180.us-east-2.compute.amazonaws.com'

In [3]:
# grep "<http://dbkwik.webdatacommons.org/Phineas_and_Ferb_Fano/" dbkwik.nt > phineas.nt
# :%s/> .$/>/g

# REPEAT COMICS

# vocal.nt
# call.nt

pd.options.display.max_colwidth = 100
df = pd.read_csv('comics.nt', sep='\s+', names=['s', 'p', 'o'])

In [4]:
wiki = 'http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik'
ontologies = df[df['o'].str.contains(wiki + '/ontology')]['o'].unique()
properties = df[df['p'].str.contains(wiki + '/property')]['p'].unique()
resources = df[df['s'].str.contains(wiki + '/resource')]['s'].unique()

In [5]:
print(len(properties))
print(len(resources))
print(len(ontologies))
print(ontologies)

1480
131933
60
['<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Image>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Agent>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Company>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Organisation>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Comic>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Work>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/WrittenWork>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Cartoon>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/HollywoodCartoon>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/ComicStrip>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/ComicsCharacter>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/FictionalCharacter>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wi

In [11]:
def get_class_count(ontology):
    sparql = SPARQLWrapper("http://" + EC2_URI + ":3030/dbkwik/query")
    query = """
        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> """ + ontology + """ }
    """
    print(query)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    for result in results["results"]["bindings"]:
        return result['count']['value']

class_frequencies = {}
for index in range(len(ontologies)):
    class_frequencies[ontologies[index]] = get_class_count(ontologies[index])

print(class_frequencies)


        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Image> }
    

        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Agent> }
    

        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Company> }
    

        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Organisation> }
    

        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Comic> }
    

        SELECT (count(?s) as ?


        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/CareerStation> }
    

        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/TimePeriod> }
    

        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Writer> }
    

        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/PersonFunction> }
    

        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Award> }
    

        SELECT (

In [20]:
def get_property_count_by_class(prop, classes, class_frequencies):
    sparql = SPARQLWrapper("http://" + EC2_URI + ":3030/dbkwik/query")
#     query = """
#         SELECT ?class (count(?class) as ?count) WHERE { 
#             VALUES ?class { """ + classes + """ }
#             ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?class .
#             {
#                 select distinct ?s ?class {
#                     {?s """ + prop + """ ?o. ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?class .}
#                     UNION { ?o """ + prop + """ ?s. ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?class . }
#                 }
#             }            
#         } group by ?class
#     """
    query = """
         SELECT ?class (count(?class) as ?count) WHERE {              
             select distinct ?s ?class {
                 VALUES ?class { """ + classes + """ }
                 {?s """ + prop + """ ?o. ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?class .}
                 UNION { ?o """ + prop + """ ?s. ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?class . }
             }
         } group by ?class    
    """
#     print(query)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    output = {}
    for result in results["results"]["bindings"]:
        if 'class' in result:
            output['<' + result['class']['value'] + '>'] = int(result['count']['value']) / int(class_frequencies['<' + result['class']['value'] + '>'])
    return output
        
classes = ' '.join(ontologies)
property_frequencies = {}

length = len(properties)
for index in range(length):
    property_frequencies[properties[index]] = get_property_count_by_class(properties[index], classes, class_frequencies)
    print(properties[index], index, length, ' DONE')

# Save to file    
with open("props.nt", "w") as file:
    for prop in property_frequencies:
        for ont in property_frequencies[prop]:
            file.write(prop + ' ' + ont + ' "' + str(property_frequencies[prop][ont]) + '"^^<http://www.w3.org/2001/XMLSchema#double> . \n')
print('File Saved')

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/colourist> 0 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/coverartist> 1 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/editor> 2 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/executiveEditor> 3 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/inker> 4 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/issuelist> 5 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/letterer> 6 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/penciler> 7 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/rating> 8 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/volume> 9 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/writer> 10 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wi

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/headtitle> 94 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/event> 95 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/tpb> 96 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/featured> 97 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/horror> 98 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/imagesize> 99 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/distinguish> 100 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/first> 101 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/heroes> 102 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/historytext> 103 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/last> 104 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/affiliation> 188 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/pseudonyms> 189 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/professionalhistory> 190 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/alignment> 191 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/education> 192 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/eyes> 193 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/hair> 194 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/identity> 195 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/maritalstatus> 196 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/weaknesses> 197 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/image4text> 198 1480  DONE
<http://dbkwi

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/music> 280 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/screenplay> 281 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/starring> 282 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/years> 283 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/additionaldesigners> 284 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/currentowner> 285 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/leaddesigner> 286 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/placeofcreation> 287 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/previousowners> 288 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/production> 289 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/audioFormat> 290 1480  DONE

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/category> 372 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/issn> 373 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/lastdate> 374 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/storyarc> 375 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/cast> 376 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/coverartst> 377 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/letrerer> 378 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/president> 379 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/upc> 380 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/cco> 381 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/wrtier> 382 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics

...

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/origissues> 1025 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/historyheader> 1026 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/caricaturist> 1027 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/technothril> 1028 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/otherdenizens> 1029 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/aproducer> 1030 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/eproducer> 1031 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/habitat> 1032 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/viewers> 1033 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/artwork> 1034 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/steampunk> 1035 1480  DONE
<h

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/ancestor> 1118 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/ethnicity> 1119 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/familycolor> 1120 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/glotto> 1121 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/glottorefname> 1122 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/iso> 1123 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/lingua> 1124 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/nativename> 1125 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/notice> 1126 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/pronunciation> 1127 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/speakers> 1128 1480  DONE
<http://dbkwik.w

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/custom> 1209 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/d> 1210 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/l> 1211 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/q> 1212 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/v> 1213 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/z> 1214 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/4a> 1215 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/4p> 1216 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/4y> 1217 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/5a> 1218 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/5p> 1219 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/5y> 1220 1480  DONE
<http

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/separator> 1301 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/portrayedBy> 1302 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/tpb!> 1303 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/firstissue> 1304 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/lastissue> 1305 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/pseudonym> 1306 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/originalpublisher> 1307 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/inUniverse> 1308 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/residence> 1309 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/runningTime> 1310 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/cyberpunk> 1311 1480  DONE
<

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/purpose> 1393 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/issues(vol.1)> 1394 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/issues(vol.2)> 1395 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/row6Info> 1396 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/writingCredits> 1397 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/loeg> 1398 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/bond> 1399 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/circulationYear> 1400 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/totalCirculation> 1401 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/rev5score> 1402 1480  DONE
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/cleanup> 1403 1480  DONE

In [ ]:
property_frequencies = {}
def get_property_count(prop, resource_count):
    sparql = SPARQLWrapper("http://" + EC2_URI + ":3030/dbkwik/query")
    query = """
        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s """ + prop + """ ?o .            
        }
    """
    print(query)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    for result in results["results"]["bindings"]:
        return int(result['count']['value']) / resource_count
    return 0

for index in range(len(properties)):
    property_frequencies[properties[index]] = get_property_count(properties[index], len(resources))
    print(properties[index], property_frequencies[properties[index]])

# Save to File    
with open("props_rank.nt", "w") as file:
    count = 0
    length = len(property_frequencies)
    for prop in property_frequencies:
        file.write(prop + ' <http://purl.org/voc/vrank#proprank> ' + '"' + str(property_frequencies[prop]) + '"^^<http://www.w3.org/2001/XMLSchema#double> . \n')
        print(count, length)
        count += 1
print('File Saved')


        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/colourist> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/colourist> 0.20218595802414863

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/coverartist> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/coverartist> 0.2988638172405691

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/editor> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/editor> 0.18805757467805628

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/executiveEditor> ?o .            
        }
    
<http:/

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/month> 0.4067140139313136

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/nextissue> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/nextissue> 0.037420508894666235

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/notes> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/notes> 0.11652126458126473

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/originalprice> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/originalprice> 0.12723124616282508

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacomm

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/companyLogo> 0.00046993549756315706

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/companyType> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/companyType> 0.0006063683839524607

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/defunct> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/defunct> 0.0005912091743536492

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/foundation> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/foundation> 0.0010156670431203717

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://d

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/nextvol> 0.00040171905436850523

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/headtitle> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/headtitle> 0.013324945237355324

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/event> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/event> 0.02436084982529011

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/tpb> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/tpb> 0.0027513965421842907

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/logo> 0.00037898023997028795

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/owner> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/owner> 0.00019706972478454973

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/1a> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/1a> 0.00012127367679049214

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/1p> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/1p> 0.00010611446719168063

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_W

...

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/box1Text> 1.5159209598811518e-05

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/box2Image> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/box2Image> 1.5159209598811518e-05

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/box2Text> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/box2Text> 1.5159209598811518e-05

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/box3Image> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/box3Image> 1.5159209598811518e-05

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwi

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/finalnumber> 1.5159209598811518e-05

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/executivceEditor> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/executivceEditor> 7.579604799405759e-06

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/storoytitle> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/storoytitle> 7.579604799405759e-06

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/recommendedReading> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/recommendedReading> 7.579604799405759e-06

        SELECT (COUNT(DISTINCT ?s) as ?count)

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/originalpublisher> 7.579604799405759e-06

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/inUniverse> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/inUniverse> 7.579604799405759e-06

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/residence> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/residence> 1.5159209598811518e-05

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/runningTime> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/runningTime> 3.0318419197623035e-05

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/gamepro> 1.5159209598811518e-05

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/breed> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/breed> 7.579604799405759e-06

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/dislikes> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/dislikes> 7.579604799405759e-06

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/likes> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/likes> 7.579604799405759e-06

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org

<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/aka> 1.5159209598811518e-05

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/prodWebsite> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/prodWebsite> 1.5159209598811518e-05

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/supProducer> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/supProducer> 1.5159209598811518e-05

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/startyr+> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/property/startyr+> 7.579604799405759e-06

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbk